In [1]:
#Loading important libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
#np.savez('data1.npz', X = datars, y = main_y)
loadfile = np.load("data1.npz")
datars = loadfile["X"]
y = loadfile["y"]
del loadfile

In [29]:
#Preparing main dataset
datars = datars.reshape(datars.shape[0], -1)

print(datars.shape)
print(y.shape)
main_dataset = np.concatenate((datars, y), axis = 1)
print(main_dataset.shape)

#Defining main x and y for training
main_x = main_dataset[:, :-1]
main_y = main_dataset[:, -1]
print(main_x.shape)
print(main_y.shape)

(1875, 21411)
(1875, 1)
(1875, 21412)
(1875, 21411)
(1875,)


In [30]:
# event_id = {
#     "taskstart" : '9',
#     "cue" : "1",
#     "go" : "2",
#     "button press" : "5",
#     "no-go" : "4",
#     "task end": "10",
#     "error 1" : "3",
#     "error 2" : "6",
#     "error 3" : "7",
#     "error 4" : "8",
#     "error 5" : "11"
# }
# event_id

In [31]:
#Removing events which are not needed for current task
required_events = [2, 5]    #Events to classify

dataset = main_dataset[np.array(list(r[-1] in required_events for r in main_dataset))]

print(dataset.shape)

#Defining x and y
x = dataset[:, :-1]
y = dataset[:, -1]
print(x.shape)
print(y.shape)
print(list(set(y)))

permutations = np.random.permutation(len(x))
# print(permutation)
print(x[-1])
x = x[permutations]
y = y[permutations]
print(x[-1])

(957, 21412)
(957, 21411)
(957,)
[2.0, 5.0]
[ 9.89868760e-07  1.28943692e-06  1.08217038e-06 ... -1.01997355e-05
 -1.10463195e-05 -1.12842527e-05]
[-1.62371865e-07 -2.82117731e-06  1.34060053e-06 ...  6.76570345e-06
  6.34351181e-07 -1.50353894e-07]


In [35]:
#Encoding y
dict = {}
for i in range(len(required_events)):
    dict[required_events[i]] = i
    
for i in range(len(y)):
    y[i] = dict[y[i]]

print(list(set(y)))

[0.0, 1.0]


### Other Models

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [37]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression
Logclassifier = LogisticRegression(random_state = 0, max_iter = 1000)
Logclassifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [39]:
y_pred_log = Logclassifier.predict(X_test)

In [40]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_log)
print(cm)

[[86 10]
 [10 86]]


In [41]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = Logclassifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 89.41 %
Standard Deviation: 2.17 %


#### Kernal SVM

In [42]:
from sklearn.svm import SVC
SVCclassifier = SVC(kernel = 'rbf', random_state = 0)
SVCclassifier.fit(X_train, y_train)

SVC(random_state=0)

In [43]:
y_pred_SVC = SVCclassifier.predict(X_test)

In [44]:
y_pred_SVC

array([1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1.])

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_SVC)
print(cm)
accuracy_score(y_test, y_pred_SVC)

[[86 10]
 [16 80]]


0.8645833333333334

#### Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier
RFclassifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
RFclassifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [47]:
y_pred_RF = RFclassifier.predict(X_test)

In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_RF)
print(cm)
accuracy_score(y_test, y_pred_RF)

[[79 17]
 [33 63]]


0.7395833333333334

#### KNN

In [49]:
from sklearn.neighbors import KNeighborsClassifier
KNNclassifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
KNNclassifier.fit(X_train, y_train)

KNeighborsClassifier()

In [50]:
y_pred_KNN = KNNclassifier.predict(X_test)

In [51]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_KNN)
print(cm)
accuracy_score(y_test, y_pred_KNN)

[[72 24]
 [29 67]]


0.7239583333333334

#### Naive Bayes

In [52]:
from sklearn.naive_bayes import GaussianNB
NBclassifier = GaussianNB()
NBclassifier.fit(X_train, y_train)

GaussianNB()

In [53]:
y_pred_NB = NBclassifier.predict(X_test)

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_NB)
print(cm)
accuracy_score(y_test, y_pred_NB)

[[81 15]
 [31 65]]


0.7604166666666666